In [70]:
print("Hello World!")

Hello World!


In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark1 = SparkSession.builder.appName("PracticeApp").master("local[*]").getOrCreate()

print("Spark Session for Practice is Created")
print("Spark version: ", spark1.version)
print("------------- Happy Codding ----------")

Spark Session for Practice is Created
Spark version:  4.0.1
------------- Happy Codding ----------


In [4]:
data = [
    {"name":"masud rana", "cgpa":3.60, "address":"Kanchan, Rupganj, Narayanganj", "university":"Green University of Bangladesh"}
]

df = spark1.createDataFrame(data)
df.show(truncate=False)


+-----------------------------+----+----------+------------------------------+
|address                      |cgpa|name      |university                    |
+-----------------------------+----+----------+------------------------------+
|Kanchan, Rupganj, Narayanganj|3.6 |masud rana|Green University of Bangladesh|
+-----------------------------+----+----------+------------------------------+



## Dwonload 'csv' data from kaggle

In [ ]:
import kagglehub
import shutil
from pathlib import Path

downloaded_dir = kagglehub.dataset_download("karkavelrajaj/amazon-sales-dataset", force_download=True)
source_dir = Path(downloaded_dir)

if not source_dir.exists() and not source_dir.is_dir():
    raise FileNotFoundError("Maybe kaggle unable to download the dataset!")

files = [f for f in source_dir.iterdir() if f.is_file()]

if len(files) != 1:
    raise RuntimeError("It should contain exactly one file!")

source_file = files[0]
source_file_ext = source_file.suffix
print("source file path and extentions: ", source_file, source_file_ext)


project_root = Path.cwd()
target_dir = project_root / "data" / "raw"
target_dir.mkdir(parents=True, exist_ok=True)
print("project root: ", project_root, "target dir: ", target_dir)

target_file_name = f"amazon_sales_raw{source_file_ext}"
target_file_path = target_dir / target_file_name
print("target file name: ", target_file_name, "target file path: ", target_file_path)
shutil.move(str(source_file), str(target_file_path)) 

c:\Users\User\PycharmProjects\AmazonSalesETLwithAirflow\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.95M/1.95M [00:01<00:00, 1.29MB/s]

Extracting files...
source file path and extentions:  C:\Users\User\.cache\kagglehub\datasets\karkavelrajaj\amazon-sales-dataset\versions\1\amazon.csv .csv
project root:  c:\Users\User\PycharmProjects\AmazonSalesETLwithAirflow target dir:  c:\Users\User\PycharmProjects\AmazonSalesETLwithAirflow\data\raw
target file name:  amazon_sales_raw.csv target file path:  c:\Users\User\PycharmProjects\AmazonSalesETLwithAirflow\data\raw\amazon_sales_raw.csv


'c:\\Users\\User\\PycharmProjects\\AmazonSalesETLwithAirflow\\data\\raw\\amazon_sales_raw.csv'

### Read csv file

In [118]:
# print schema data types
from pathlib import Path
target_file_path = Path.cwd() / "data" / "raw"
df = (
    spark1.read
    .option("header", "true")
    .option("multiLine", "true")
    .option("inferSchema", "true")
    .option("quote", '"')
    .option("escape", '"')
    .option("mode", "PERMISSIVE")
    .option("columnNameOfCorruptRecord", "_corrupt_record")
    .csv(str(target_file_path))
)
df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- discounted_price: string (nullable = true)
 |-- actual_price: string (nullable = true)
 |-- discount_percentage: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_count: string (nullable = true)
 |-- about_product: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_title: string (nullable = true)
 |-- review_content: string (nullable = true)
 |-- img_link: string (nullable = true)
 |-- product_link: string (nullable = true)



In [106]:
df.count()

1465

In [7]:
# print all columns
print(len(df.columns))

df.columns

16


['product_id',
 'product_name',
 'category',
 'discounted_price',
 'actual_price',
 'discount_percentage',
 'rating',
 'rating_count',
 'about_product',
 'user_id',
 'user_name',
 'review_id',
 'review_title',
 'review_content',
 'img_link',
 'product_link']

In [14]:
df.show(10)

+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|product_id|        product_name|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|        about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|B07JW9H4J1|Wayona Nylon Brai...|Computers&Accesso...|            ₹399|      ₹1,099|                64%|   4.2|      2

In [15]:
# count of non-null values in each column
from pyspark.sql.functions import col, count

df.select([count(c).alias(c) for c in df.columns]).show()

+----------+------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+
|product_id|product_name|category|discounted_price|actual_price|discount_percentage|rating|rating_count|about_product|user_id|user_name|review_id|review_title|review_content|img_link|product_link|
+----------+------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+
|      1465|        1465|    1465|            1465|        1465|               1465|  1465|        1463|         1465|   1465|     1465|     1465|        1465|          1465|    1465|        1465|
+----------+------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+



In [16]:
# count null values in each column
from pyspark.sql.functions import col, sum, when

df.select(
    [sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df.columns]
).show()

+----------+------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+
|product_id|product_name|category|discounted_price|actual_price|discount_percentage|rating|rating_count|about_product|user_id|user_name|review_id|review_title|review_content|img_link|product_link|
+----------+------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+
|         0|           0|       0|               0|           0|                  0|     0|           2|            0|      0|        0|        0|           0|             0|       0|           0|
+----------+------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+



In [119]:
# rename the product_name to product_title
df = df.withColumnRenamed("product_name", "product_title")

In [120]:
df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- category: string (nullable = true)
 |-- discounted_price: string (nullable = true)
 |-- actual_price: string (nullable = true)
 |-- discount_percentage: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_count: string (nullable = true)
 |-- about_product: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_title: string (nullable = true)
 |-- review_content: string (nullable = true)
 |-- img_link: string (nullable = true)
 |-- product_link: string (nullable = true)



## verify every column is correct

In [121]:
# check other values interefare with discounted_price column
from pyspark.sql.functions import col

invalid_discounted_price_df = df.filter(
    col("discounted_price").isNotNull() &
    ~col("discounted_price").rlike(r"^₹[\d,]+(\.\d+)?$")
)

invalid_discounted_price_df.select("discounted_price").show(truncate=False)


+----------------+
|discounted_price|
+----------------+
+----------------+



In [122]:
# remove special characters and cast  
from pyspark.sql.functions import when, regexp_replace, col

def remove_symbol_and_cast(column_name, target_type):
    cleaned_symbol = regexp_replace(col(column_name), "[^\d.]", "")
    return when(
        cleaned_symbol.rlike(r"^\d+(\.\d+)?$"),
        cleaned_symbol.try_cast(target_type)
    ).otherwise(None)


df = df.withColumn("discounted_price", remove_symbol_and_cast("discounted_price", "float")) \
    .withColumn("actual_price", remove_symbol_and_cast("actual_price", "float")) \
    .withColumn("discount_percentage", remove_symbol_and_cast("discount_percentage", "int")) \
    .withColumn("rating_count", remove_symbol_and_cast("rating_count", "int"))

In [123]:
df.show(5)

+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|product_id|       product_title|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|        about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|B07JW9H4J1|Wayona Nylon Brai...|Computers&Accesso...|           399.0|      1099.0|                 64|   4.2|       

In [77]:
df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- discounted_price: float (nullable = true)
 |-- actual_price: float (nullable = true)
 |-- discount_percentage: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_count: integer (nullable = true)
 |-- about_product: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_title: string (nullable = true)
 |-- review_content: string (nullable = true)
 |-- img_link: string (nullable = true)
 |-- product_link: string (nullable = true)



In [124]:
df = df.withColumn("rating", col("rating").try_cast("float"))

In [125]:
df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- category: string (nullable = true)
 |-- discounted_price: float (nullable = true)
 |-- actual_price: float (nullable = true)
 |-- discount_percentage: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- rating_count: integer (nullable = true)
 |-- about_product: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_title: string (nullable = true)
 |-- review_content: string (nullable = true)
 |-- img_link: string (nullable = true)
 |-- product_link: string (nullable = true)



In [126]:
# see null values in each columns
df.select(
    [sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df.columns]
).show()

+----------+-------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+
|product_id|product_title|category|discounted_price|actual_price|discount_percentage|rating|rating_count|about_product|user_id|user_name|review_id|review_title|review_content|img_link|product_link|
+----------+-------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+
|         0|            0|       0|               0|           0|                  0|     1|           2|            0|      0|        0|        0|           0|             0|       0|           0|
+----------+-------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+



In [127]:
# drop rows will null values
df = df.dropna(subset=[c for c in df.columns])

df.count()

1462

In [128]:
# see null values in each columns
df.select(
    [sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df.columns]
).show()

+----------+-------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+
|product_id|product_title|category|discounted_price|actual_price|discount_percentage|rating|rating_count|about_product|user_id|user_name|review_id|review_title|review_content|img_link|product_link|
+----------+-------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+
|         0|            0|       0|               0|           0|                  0|     0|           0|            0|      0|        0|        0|           0|             0|       0|           0|
+----------+-------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+



In [129]:
df.count()

1462

In [130]:
df.show(10)

+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|product_id|       product_title|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|        about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|B07JW9H4J1|Wayona Nylon Brai...|Computers&Accesso...|           399.0|      1099.0|                 64|   4.2|       

In [131]:
# count and sum of numeric columns
from pyspark.sql.functions import col, count, sum, round as spark_round
from pyspark.sql.types import IntegerType, FloatType, DoubleType, LongType, ShortType

# filter only numeric columns
numeric_types = (IntegerType, FloatType, DoubleType, LongType, ShortType)
numeric_columns = [
    field.name
    for field in df.schema.fields
    if isinstance(field.dataType, numeric_types)
]

numeric_columns

['discounted_price',
 'actual_price',
 'discount_percentage',
 'rating',
 'rating_count']

In [132]:
# count of numeric columns
df.select([count(c).alias(c) for c in numeric_columns]).show()

# sum of each numeric columns
df.select([spark_round(sum(col(c)), 3).alias(c) for c in numeric_columns]).show()

+----------------+------------+-------------------+------+------------+
|discounted_price|actual_price|discount_percentage|rating|rating_count|
+----------------+------------+-------------------+------+------------+
|            1462|        1462|               1462|  1462|        1462|
+----------------+------------+-------------------+------+------------+

+----------------+------------+-------------------+------+------------+
|discounted_price|actual_price|discount_percentage|rating|rating_count|
+----------------+------------+-------------------+------+------------+
|      4576033.43|  7972414.28|              69697|5989.4|    26765385|
+----------------+------------+-------------------+------+------------+



In [133]:
df.show(5)

+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|product_id|       product_title|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|        about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|B07JW9H4J1|Wayona Nylon Brai...|Computers&Accesso...|           399.0|      1099.0|                 64|   4.2|       

## Transformation apply

In [134]:
# add price difference column
df = df.withColumn("price_difference", col("actual_price") - col("discounted_price"))

df.show(5)

+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+
|product_id|       product_title|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|        about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|price_difference|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+
|B07JW9H4J1|Wayona Nylon Brai...|Computers&Accesso...|           39

In [135]:
# rating bucket
df = df.withColumn(
    "rating_bucket",
    when(col("rating") >= 4.5, "Excellent")
    .when(col("rating") >= 4.0, "Good")
    .when(col("rating") >= 3.0, "Average")
    .otherwise("Poor")
)

df.show(10)

+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+
|product_id|       product_title|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|        about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|price_difference|rating_bucket|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+
|B07JW9H4J1|Wayona Nylon 

In [136]:
# review strength bucket
df = df.withColumn(
    "rating_strength",
    when(col("rating_count") >= 10000, "Very High")
    .when(col("rating_count") >= 1000, "High")
    .when(col("rating_count") >= 100, "Medium")
    .otherwise("Low")
)

In [137]:
df.show(10)

+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+---------------+
|product_id|       product_title|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|        about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|price_difference|rating_bucket|rating_strength|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------

In [138]:
# discount percentage bucket
from pyspark.sql.functions import when

df = df.withColumn(
    "discount_bucket",
    when(col("discount_percentage") >= 80, "Very High Discount")
    .when(col("discount_percentage") >= 50, "High Discount")
    .when(col("discount_percentage") >= 20, "Medium Discount")
    .otherwise("Low Discount")
)

In [139]:
df.show(10)

+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+---------------+------------------+
|product_id|       product_title|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|        about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|price_difference|rating_bucket|rating_strength|   discount_bucket|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------

In [140]:
# column validation
from pyspark.sql.functions import col

def column_validation(df, column_name):
    null_count = df.filter(col(column_name).isNull()).count()
    if null_count == 0:
        print(f"Column: {column_name}, Nul= No")
    elif null_count > 1:
        print(f"Column: {column_name} Null = {null_count}")

for column in df.columns:
    column_validation(df, column)


Column: product_id, Nul= No
Column: product_title, Nul= No
Column: category, Nul= No
Column: discounted_price, Nul= No
Column: actual_price, Nul= No
Column: discount_percentage, Nul= No
Column: rating, Nul= No
Column: rating_count, Nul= No
Column: about_product, Nul= No
Column: user_id, Nul= No
Column: user_name, Nul= No
Column: review_id, Nul= No
Column: review_title, Nul= No
Column: review_content, Nul= No
Column: img_link, Nul= No
Column: product_link, Nul= No
Column: price_difference, Nul= No
Column: rating_bucket, Nul= No
Column: rating_strength, Nul= No
Column: discount_bucket, Nul= No


In [141]:
df.show(3)

+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+---------------+------------------+
|product_id|       product_title|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|        about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|price_difference|rating_bucket|rating_strength|   discount_bucket|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------

In [46]:
df.groupBy(["product_id"]).count().filter("count > 1").count()

92

In [35]:
df.groupBy(["user_id"]).count().filter("count > 1").count()

144

In [36]:
df.groupBy(["user_name"]).count().filter("count > 1").count()

144

In [37]:
df.groupBy(["review_id"]).count().filter("count > 1").count()

144

In [38]:
df.groupBy(["review_title"]).count().filter("count > 1").count()

144

In [39]:
df.groupBy(["product_id", "user_id", "review_id"]).count().filter("count > 1").count()

84

In [40]:
df.groupBy(["product_id", "user_id", "user_name"]).count().filter("count > 1").count()

84

In [41]:
df.groupBy(["product_id", "user_id", "review_id", "rating", "discounted_price"]).count().filter("count > 1").count()

82

In [42]:
# duplicate check
df.groupBy(df.columns).count().filter("count > 1").show(4)

+----------+------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+----------------+-------------+---------------+---------------+-----+
|product_id|product_name|category|discounted_price|actual_price|discount_percentage|rating|rating_count|about_product|user_id|user_name|review_id|review_title|review_content|img_link|product_link|price_difference|rating_bucket|review_strength|discount_bucket|count|
+----------+------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+----------------+-------------+---------------+---------------+-----+
+----------+------------+--------+----------------+------------+-------------------+------+------------+-------------+-------+---------+---------+------------+--------------+--------+------------+------

In [48]:
print(df.count())
df.show(2)

1462
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+---------------+---------------+
|product_id|        product_name|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|       about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|price_difference|rating_bucket|rating_strength|discount_bucket|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------

### save cleaned data

In [142]:
from pathlib import Path

project_root = Path.cwd()
cleaned_data_dir = project_root / "data" / "processed"
cleaned_data_dir.mkdir(parents=True, exist_ok=True)

cleaned_data_path = cleaned_data_dir / "amazon_sales_cleaned"

print("Total cleaned data: ", df.count())
df.write \
    .mode("overwrite") \
    .parquet(str(cleaned_data_path))


Total cleaned data:  1462


#### The dataset is product centric 
This table is product-centric. Each row represents a product snapshot. Review and user fields are descriptive metadata and not unique identifiers
- as product id is not unique
- user_id/name or review_id/title is not atomic
- demormalized dataset

## Gold Layer

In [143]:
from pathlib import Path

project_root = Path.cwd()
cleaned_data_dir = project_root / "data" / "processed"
cleaned_data_dir.mkdir(parents=True, exist_ok=True)

cleaned_data_path = cleaned_data_dir / "amazon_sales_cleaned"

cleaned_df = spark1.read.parquet(str(cleaned_data_path))

print(cleaned_df.count())
cleaned_df.show(20)

1462
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+---------------+------------------+
|product_id|       product_title|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|        about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|price_difference|rating_bucket|rating_strength|   discount_bucket|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+---------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---

In [144]:
# column validation
from functools import reduce
from pyspark.sql.functions import col

def column_validation(df, column_name):
    null_count = df.filter(col(column_name).isNull()).count()
    if null_count == 0:
        print(f"Column: {column_name}, Nul= No")
    elif null_count > 1:
        print(f"Column: {column_name} Null = {null_count}")

for column in cleaned_df.columns:
    column_validation(cleaned_df, column)


Column: product_id, Nul= No
Column: product_title, Nul= No
Column: category, Nul= No
Column: discounted_price, Nul= No
Column: actual_price, Nul= No
Column: discount_percentage, Nul= No
Column: rating, Nul= No
Column: rating_count, Nul= No
Column: about_product, Nul= No
Column: user_id, Nul= No
Column: user_name, Nul= No
Column: review_id, Nul= No
Column: review_title, Nul= No
Column: review_content, Nul= No
Column: img_link, Nul= No
Column: product_link, Nul= No
Column: price_difference, Nul= No
Column: rating_bucket, Nul= No
Column: rating_strength, Nul= No
Column: discount_bucket, Nul= No


### products gold table

In [145]:
# products gold table
from pyspark.sql.functions import col, avg, max, min, first, round as spark_round

products_gold_df = (
    cleaned_df
    .groupBy("product_id")
    .agg(
        first("product_title", ignorenulls=True).alias("product_title"),
        first("category", ignorenulls=True).alias("category"),
        spark_round(avg("rating"), 2).alias("avg_rating"),
        max("rating_count").alias("rating_count"),
        min("discounted_price").alias("discounted_price"),
        first("actual_price", ignorenulls=True).alias("actual_price"),
        first("discount_percentage", ignorenulls=True).alias("discount_percentage"),
        first("price_difference", ignorenulls=True).alias("price_difference"),
        first("rating_bucket", ignorenulls=True).alias("rating_bucket"),
        first("rating_strength", ignorenulls=True).alias("rating_strength")
    )
)

print(products_gold_df.count())
products_gold_df.show(10)

1348
+----------+--------------------+--------------------+----------+------------+----------------+------------+-------------------+----------------+-------------+---------------+
|product_id|       product_title|            category|avg_rating|rating_count|discounted_price|actual_price|discount_percentage|price_difference|rating_bucket|rating_strength|
+----------+--------------------+--------------------+----------+------------+----------------+------------+-------------------+----------------+-------------+---------------+
|B002PD61Y4|D-Link DWA-131 30...|Computers&Accesso...|       4.1|        8131|           507.0|      1208.0|                 58|           701.0|         Good|           High|
|B002SZEOLG|TP-Link Nano USB ...|Computers&Accesso...|       4.2|      179692|           749.0|      1339.0|                 44|           590.0|         Good|      Very High|
|B003B00484|Duracell Plus AAA...|Electronics|Gener...|       4.3|       27201|           399.0|       499.0|       

In [146]:
# save into data folder
project_root = Path.cwd()
cleaned_data_dir = project_root / "data" / "gold"
cleaned_data_dir.mkdir(parents=True, exist_ok=True)

amazon_gold_data = cleaned_data_dir / "gold_products_summary"

products_gold_df.write.mode("overwrite").parquet(str(amazon_gold_data))


In [48]:
from pyspark.sql.utils import AnalysisException
from py4j.protocol import Py4JJavaError
jdbc_url = "jdbc:postgresql://localhost:5432/airflow"

props = {
    "user": "airflow",
    "password": "airflow"
}
 
try:
    df.write \
      .mode("append") \
      .jdbc(
          url=jdbc_url,
          table="product_summary",
          properties=props
      )
    print("Data successfully written to product_summary")

except AnalysisException as e:
    print("ANALYSIS EXCEPTION")
    print(e)

except Py4JJavaError as e:
    print("JAVA ERROR (root cause below)")
    print(e.java_exception) 

except Exception as e:
    print("UNKNOWN ERROR")
    print(e)


ANALYSIS EXCEPTION
[COLUMN_NOT_DEFINED_IN_TABLE] "FLOAT" column `rating` is not defined in table `product_summary`, defined table columns are: `product_id`, `product_name`, `category`, `discounted_price`, `actual_price`, `discount_percentage`, `rating`, `rating_count`, `about_product`, `user_id`, `user_name`, `review_id`, `review_title`, `review_content`, `img_link`, `product_link`, `price_difference`, `rating_bucket`, `review_strength`, `discount_bucket`. SQLSTATE: 42703


In [147]:
spark1._jvm.java.lang.Class.forName("org.postgresql.Driver")
print("PostgreSQL JDBC driver is available")


PostgreSQL JDBC driver is available


### category gold table

In [148]:
cleaned_df.show(1)

+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-------------+---------------+---------------+
|product_id|       product_title|            category|discounted_price|actual_price|discount_percentage|rating|rating_count|       about_product|             user_id|           user_name|           review_id|        review_title|      review_content|            img_link|        product_link|price_difference|rating_bucket|rating_strength|discount_bucket|
+----------+--------------------+--------------------+----------------+------------+-------------------+------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------

In [149]:
from pyspark.sql.functions import avg, count, round as spark_round

category_gold_df = cleaned_df.groupBy("category").agg(
    count("product_id").alias("total_products"),
    spark_round(avg("rating"), 2).alias("avg_rating"),
    spark_round(avg("rating_count"), 0).alias("avg_rating_count"),
    spark_round(avg("discount_percentage"), 2).alias("avg_discount_percentage"),
    spark_round(avg("price_difference"), 2).alias("avg_price_difference")
)

print(category_gold_df.count())
print(category_gold_df.printSchema())
category_gold_df.show(2)

211
root
 |-- category: string (nullable = true)
 |-- total_products: long (nullable = false)
 |-- avg_rating: double (nullable = true)
 |-- avg_rating_count: double (nullable = true)
 |-- avg_discount_percentage: double (nullable = true)
 |-- avg_price_difference: double (nullable = true)

None
+--------------------+--------------+----------+----------------+-----------------------+--------------------+
|            category|total_products|avg_rating|avg_rating_count|avg_discount_percentage|avg_price_difference|
+--------------------+--------------+----------+----------------+-----------------------+--------------------+
|Computers&Accesso...|             1|       4.1|          7333.0|                   75.0|               600.0|
|OfficeProducts|Of...|             4|       4.4|          7130.0|                   5.75|                8.75|
+--------------------+--------------+----------+----------------+-----------------------+--------------------+
only showing top 2 rows


In [150]:
project_root = Path.cwd()
gold_data_dir = project_root / "data" / "gold"
gold_data_dir.mkdir(parents=True, exist_ok=True)
category_gold_data_path = gold_data_dir / "gold_category_summary"

category_gold_df.write.mode("overwrite").parquet(str(category_gold_data_path))


In [151]:
category_gold_df.printSchema()

root
 |-- category: string (nullable = true)
 |-- total_products: long (nullable = false)
 |-- avg_rating: double (nullable = true)
 |-- avg_rating_count: double (nullable = true)
 |-- avg_discount_percentage: double (nullable = true)
 |-- avg_price_difference: double (nullable = true)



### Top products performance gold table

In [153]:
from pyspark.sql.window import Window
from pyspark.sql.functions import when, col, dense_rank

from pyspark.sql.functions import when, col

review_strength_score = when(col("rating_count") >= 10000, 4) \
    .when(col("rating_count") >= 1000, 3) \
    .when(col("rating_count") >= 100, 2) \
    .otherwise(1)

rating_bucket_score = when(col("rating") >= 4.5, 4) \
    .when(col("rating") >= 4.0, 3) \
    .when(col("rating") >= 3.0, 2) \
    .otherwise(1)

window_spec = Window.partitionBy("category").orderBy(
    col("rating_count").desc(),
    col("review_strength_score").desc(),
    col("rating_bucket_score").desc()
)


top_products_gold_df = (
    cleaned_df
    .withColumn("review_strength_score", review_strength_score)
    .withColumn("rating_bucket_score", rating_bucket_score)
    .withColumn("rank", dense_rank().over(window_spec))
    .filter(col("rank") <= 5)
    .select(
        "product_id",
        "product_title",
        "category",
        "rating",
        "rating_count",
        "discounted_price",
        "rank"
    )
)

print(top_products_gold_df.count())
top_products_gold_df.show(10)

635
+----------+--------------------+--------------------+------+------------+----------------+----+
|product_id|       product_title|            category|rating|rating_count|discounted_price|rank|
+----------+--------------------+--------------------+------+------------+----------------+----+
|B0912WJ87V|Reffair AX30 [MAX...|Car&Motorbike|Car...|   3.8|        1118|          2339.0|   1|
|B097C564GC|rts [2 Pack] Mini...|Computers&Accesso...|   4.3|        4426|           294.0|   1|
|B094DQWV9B|Kanget [2 Pack] T...|Computers&Accesso...|   4.0|        1540|           149.0|   2|
|B009LJ2BXA|Hp Wired On Ear H...|Computers&Accesso...|   3.5|        7222|           649.0|   1|
|B08SCCG9D4|JBL Commercial CS...|Computers&Accesso...|   3.9|       14969|           949.0|   1|
|B08HD7JQHX|HUMBLE Dynamic La...|Computers&Accesso...|   3.3|        2804|           199.0|   2|
|B08K9PX15C|Zebronics Zeb Won...|Computers&Accesso...|   4.0|        7352|           849.0|   1|
|B08SBH499M|ZEBRONICS Zeb-

In [154]:
project_root = Path.cwd()
gold_data_dir = project_root / "data" / "gold"
gold_data_dir.mkdir(parents=True, exist_ok=True)
category_gold_data_path = gold_data_dir / "gold_top_products"

top_products_gold_df.write.mode("overwrite").parquet(str(category_gold_data_path))

In [155]:
top_products_gold_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- category: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- rating_count: integer (nullable = true)
 |-- discounted_price: float (nullable = true)
 |-- rank: integer (nullable = false)



### Discount gold tables

In [156]:
from pyspark.sql.functions import col, avg, round as spark_round
discount_effectiveness_gold_df = cleaned_df.groupBy("discount_bucket") \
    .agg(
        spark_round(avg(col("rating")), 2).alias("avg_rating"),
        spark_round(avg(col("rating_count")), 2).alias("avg_rating_count"),
        count(col("product_id")).alias("total_products")
    )

print(discount_effectiveness_gold_df.count())
discount_effectiveness_gold_df.show(5)

4
+------------------+----------+----------------+--------------+
|   discount_bucket|avg_rating|avg_rating_count|total_products|
+------------------+----------+----------------+--------------+
|   Medium Discount|      4.12|        18760.24|           548|
|      Low Discount|      4.17|        14953.43|           165|
|     High Discount|      4.07|        19987.49|           654|
|Very High Discount|      4.03|         9954.12|            95|
+------------------+----------+----------------+--------------+



In [157]:
project_root = Path.cwd()
gold_data_dir = project_root / "data" / "gold"
gold_data_dir.mkdir(parents=True, exist_ok=True)
discount_effectiveness_gold_data_path = gold_data_dir / "gold_discount_effectiveness"

discount_effectiveness_gold_df.write.mode("overwrite").parquet(str(discount_effectiveness_gold_data_path))

In [158]:
discount_effectiveness_gold_df.printSchema()

root
 |-- discount_bucket: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- avg_rating_count: double (nullable = true)
 |-- total_products: long (nullable = false)

